In [1]:
from inventory.models import (
    Brand,
    Category,
    Product,
    Order
)
from django.contrib.auth.models import User

In [2]:
Product.objects.all()

<QuerySet [<Product: Men Relaxed Fit Mid Rise Blue Jeans>, <Product: Motorola Edge 50 Fusion (Marshmallow Blue, 256 GB)  (12 GB RAM)>, <Product: SAMSUNG 8 kg 5 star, Ecobubble, Digital Inverter, Fully Automatic Top Load Washing Machine Grey>, <Product: SAMSUNG 8.5 kg Semi Automatic Top Load Washing Machine Black, Grey>]>

In [3]:
from django.db.models import (
    Count,
    F,
    Sum,
    OuterRef,
    Value,
    IntegerField,
    Prefetch,
    Q,
    ExpressionWrapper,
    functions,
    CharField,
    Case,
    When,
    Func
)

In [4]:
get_product_with_total_price = Order.objects.all().annotate(total_price = Product.objects.filter(uuid = OuterRef("product__uuid")).values("price")).annotate(total_price = F("quantity") * F("total_price"))

In [5]:
get_product_with_total_price

<QuerySet [<Order: Men Relaxed Fit Mid Rise Blue Jeans>, <Order: Motorola Edge 50 Fusion (Marshmallow Blue, 256 GB)  (12 GB RAM)>, <Order: SAMSUNG 8 kg 5 star, Ecobubble, Digital Inverter, Fully Automatic Top Load Washing Machine Grey>]>

In [6]:
from django_pandas.io import read_frame

In [7]:
df = read_frame(get_product_with_total_price)

In [8]:
df.loc[:, ['customer','product','quantity','total_price']]

,customer,product,quantity,total_price
0,Mayank,Men Relaxed Fit Mid Rise Blue Jeans,2,998.0
1,Mayank,"Motorola Edge 50 Fusion (Marshmallow Blue, 256...",1,24999.0
2,Deqode,"SAMSUNG 8 kg 5 star, Ecobubble, Digital Invert...",3,58470.0


In [9]:
get_customer_orders = User.objects.filter(username = "Mayank")

In [10]:
get_customer_orders

<QuerySet [<User: Mayank>]>

In [11]:
get_customer_orders = User.objects.filter(username = "Deqode").annotate(total_orders = Order.objects.filter(customer_id = OuterRef("id")).values("customer_id"))

In [12]:
get_customer_orders.values()

<QuerySet [{'id': 1, 'password': 'pbkdf2_sha256$600000$kVPyPStYJPrXYxBPBZRHM7$YH4LKNqb8PAbB+sz/DaByWZq4n3mInPvVpVIp55NxSA=', 'last_login': datetime.datetime(2024, 7, 3, 9, 8, 12, tzinfo=datetime.timezone.utc), 'is_superuser': True, 'username': 'Deqode', 'first_name': '', 'last_name': '', 'email': 'info@deqode.com', 'is_staff': True, 'is_active': True, 'date_joined': datetime.datetime(2024, 6, 18, 6, 41, 40, tzinfo=datetime.timezone.utc), 'total_orders': 1}]>

In [42]:
get_brand_products = Brand.objects.all().annotate(
    total_products = Product.objects.filter(
        brand_id = OuterRef("uuid")
    ).values("brand_id")
)

In [43]:
get_brand_products.values('name','total_products')

<QuerySet [{'name': 'UNITED DENIM', 'total_products': 1}, {'name': 'Motorola', 'total_products': 1}, {'name': 'Samsung', 'total_products': 1}]>

In [44]:
brands_with_total_products = Brand.objects.annotate(
    total_products=Subquery(
        Product.objects.filter(brand_id=OuterRef('uuid')).values('brand_id').annotate(
            count=Count('uuid')
        ).values('count')[:1]
    )
)